## 1. Создание предсказательной модели и оценивание её качества.

Считаем данные из скачанной открытой гугл-папки

In [2]:
import os
import pandas as pd
path = r'/Users/romanova/Downloads/mini-course-public'
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            #print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                #print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            #print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                #print(dir_entry_path.split("\\")[-1])
                pass

Переведём типы движений в числа

In [3]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [24]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [25]:
def rotation_matrix(vector):
    b = np.array([0, 0, -1])
    # Нормируем
    a = (vector / np.linalg.norm(vector)).reshape(3)
    v = np.cross(a, b) # Вычисляем векторное произведение
    
    if any(v):
        c = a * b
        s = np.linalg.norm(v)
        # Преобразование к матричной форме     
        kmat = np.array([ [0, -v[2], v[1]],
                          [v[2], 0, -v[0]],
                          [-v[1], v[0], 0] ])

        return np.eye(3) + kmat + kmat @ kmat * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3)
    
def rotate(df, mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(df.gFx.tolist(), df.gFy.tolist(), df.gFz.tolist()):
        res = mat @ elem
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(Acc_arr):
    sm_arr = np.cumsum(Acc_arr).tolist()
    return sm_arr, sm_arr[-1]

In [106]:
import pandas as pd
import pandas as pd
from matplotlib import pylab as plt
from datetime import datetime
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))

        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return [-1, -1, -1, -1, -1, [-1, -1, -1]]
        
# средняя частота сбора данных
    Freq = 0.0
    try:
        df[df.columns[0]] = df[df.columns[0]].astype(float)
        Time = (df[df.columns[0]][df.shape[0]-2] - df[df.columns[0]][1])
        Freq = round( (df.shape[0] - 2) / Time )
    except:
        Time = 20
        Freq = round( (df.shape[0] - 2) / Time )
        
# обрезаем начало и конец трека
    datFram_len = len(df) 
    toCut = round(datFram_len * 0.25)
    if a == -1:
        st = toCut
    else:
        st = a
            
    if b == -1:
        end = -toCut
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1
    

#    print(name)

#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))
        
########################################################
# amplitude
########################################################

# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
#    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
#    print()
    
            
########################################################
# преобразование Фурье
########################################################

    Fs = 250 #частота сбора данных
    Min = 0
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    if len(y) == 0:
        hesitation = -1
    else:
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[1:n//2]

        yabs = abs(Y)
        xmax = frq[numpy.argmax(yabs)]
        ymax = yabs.max()

        hesitation = xmax
        if hesitation == 0:
            hesitation = -1
    
#    рисование
#    ax3.plot(frq,yabs) # plotting the spectrum
#    ax3.set_xlim([0,30])
#    ax3.set_ylim([Min,0.1])

########################################################

########################################################
# Вращаем координаты, пытаемся определить Speed и модуль перемещения
    move = []
    try:
        acc_mean = [df.gFx.mean(), df.gFy.mean(), df.gFz.mean()]
        r_mat = rotation_matrix(acc_mean)
        gx, gy, gz = rotate(df, r_mat)
        df.gFx = pd.Series(gx)
        df.gFy = pd.Series(gy)
        df.gFz = pd.Series(gz)
        
        gx = [elem / Freq ** 2 for elem in gx]
        gy = [elem / Freq ** 2 for elem in gy]
        gz = [(elem + 1) / Freq ** 2 for elem in gz]
        
        speed_arrx, speedX = integrate(gx)
        moveX, _ = integrate(speed_arrx)
        
        speed_arry, speedY = integrate(gy)
        moveY, _ = integrate(speed_arry)
        
        speed_arrz, speedZ = integrate(gz)
        moveZ, _ = integrate(speed_arrz)
        
        speed = (speedX ** 2 + speedY ** 2 + speedZ ** 2) ** 0.5
        move = [moveX[-1] - moveX[0], moveY[-1] - moveX[0], moveZ[-1] - moveX[0]]
    except:
        speed = -1
########################################################     

########################################################  
# Считаем дисперсию
    # среднее по модулю ускорения
    mean_a = Acc.mean()
    disp = 0.0
    for row in Acc:
        disp += (row - mean_a) ** 2
    disp /= (df.shape[0] - 1)
########################################################  

#    print('Амплитуда:', amplitude, ' xmax:', xmax, ' ymax:', ymax, 'speed:', speed, 'disp:', disp, 'move:', move)
    return [amplitude, xmax, ymax, speed, disp, move]

Information = []
for df, target in zip(Data,Target):
    Information.append(process_data(df, name=target))

Populating the interactive namespace from numpy and matplotlib


/Users/romanova/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [114]:
def get_prediction(info):
    ampl, xmax, ymax, speed, disp, move = info[0], info[1], info[2], info[3], info[4], info[5]
    if ampl < 1 and disp < 0.035:
        return 0 # стояние
    if ampl > 4 and disp > 0.45:
        return 2 # бег
    if 0.8 < xmax < 3.1 and abs(move[1]) > max(abs(move[0]), abs(move[2])):
        return 4 # лестница
    elif 0.9 < xmax < 2.1:
        return 1 # ходьба
    if disp < 0.25 and speed > 0.03:
        return 3 # велосипед
    return 1

In [115]:
len(Data), len(Information), len(Target)

(1196, 1196, 1196)

Некоторые амплитуды посчитаны некорректно, уберём соответствующие треки.

In [116]:
Information[82:86]

[[3.4165687999378096,
  0.8712641250396029,
  0.04710516512354914,
  0.015037696199245712,
  0.023535494037381317,
  [4.657909203466622, -91.37560592878785, 88.8873176219397]],
 [1.0515078715979422,
  0.9482011270047681,
  0.06129440618711797,
  0.01569313260301183,
  0.019379061892215884,
  [85.30926255278786, 10.534278732404157, 118.79601282465639]],
 [0.9695346215909424,
  0.8101463850709577,
  0.05141749277772742,
  0.044052636054680336,
  0.011398005765218043,
  [47.55063762791014, 389.2806928415773, -15.473736453027646]],
 [0.697107325795372,
  0.8303867995285545,
  0.03808537349017529,
  0.045433862513858,
  0.008140595229556848,
  [94.07913906144664, 410.56110586488467, -22.894152867367414]]]

In [117]:
good_indices = [i for i in range(len(Information)) if not ((math.isnan(Information[i][0]) or Information[i][0]==-1) and 
                                                    (math.isnan(Information[i][1]) or Information[i][1]==-1) and
                                                    (math.isnan(Information[i][2]) or Information[i][2]==-1) and 
                                                    (math.isnan(Information[i][3]) or Information[i][3]==-1) and
                                                    (math.isnan(Information[i][4]) or Information[i][4]==-1) )]


GoodData   = [Data[i] for i in good_indices]
GoodInfo    = [Information[i] for i in good_indices]
GoodTarget = [Target[i] for i in good_indices]

In [118]:
len(Data), len(GoodInfo), len(GoodData), len(GoodTarget)

(1196, 1195, 1195, 1195)

Создадим таблицу, содержащую номер трека, правильный ответ и предсказание.

In [119]:
prediction = [get_prediction(a) for a in GoodInfo]
answer = [get_action(target) for target in GoodTarget]

Res = pd.DataFrame({'track_num' : np.arange(len(GoodData)), 'answer' : answer, 'prediction' : prediction})
Res.head(100)

,track_num,answer,prediction
0,0,0,0
1,1,0,0
2,2,0,1
3,3,0,0
4,4,0,0
5,5,1,1
6,6,1,1
7,7,1,1
8,8,1,1
9,9,3,3


Проверим качество нашей модели на тренировочных данных (на данных из открытой папки)

In [120]:
from sklearn.metrics import accuracy_score

accuracy_score(Res['answer'], Res['prediction'])

0.5087866108786611

В тренировочной папке количество треков разных типов несбалансировано, поэтому для корректной оценки качества нашей модели будем проверять качество модели на таком же распределении треков, как и в тестовой выборке:

* 18% - стояние
* 18 % - ходьба
* 18 % - бег
* 18 % - подъем по лестнице
* 18 % - велосипед
* 10% - другие типы движений

In [121]:
Res['answer'].value_counts(), len(Res)

( 1    299
  0    284
  4    221
  2    216
 -1     62
  3     57
  5     31
  6     13
  7      8
  8      4
 Name: answer, dtype: int64, 1195)

In [122]:
import random

ind_0 = random.sample(Res[Res.answer == 0].index.tolist(), 32)
ind_1 = random.sample(Res[Res.answer == 1].index.tolist(), 32)
ind_2 = random.sample(Res[Res.answer == 2].index.tolist(), 32)
ind_3 = random.sample(Res[Res.answer == 3].index.tolist(), 32)
ind_4 = random.sample(Res[Res.answer == 4].index.tolist(), 32)
ind_5 = random.sample(Res[Res.answer == 5].index.tolist(), 6)
ind_6 = random.sample(Res[Res.answer == 6].index.tolist(), 6)
ind_7 = random.sample(Res[Res.answer == 7].index.tolist(), 6)

ind_balanced = ind_0 + ind_1 + ind_2 + ind_3 + ind_4 + ind_5 + ind_6 + ind_7 
len(ind_balanced)

178

In [123]:
Res_balanced = Res.iloc[ind_balanced]

In [125]:
accuracy_score(Res_balanced['answer'], Res_balanced['prediction'])

0.5224719101123596